# Creazione dizionario dei sinonimi

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import os
import pandas as pd
import json
import shutil

In [ ]:
def remove_and_make_dir(path):
    if os.path.exists(path):
        shutil.rmtree(path)
        os.mkdir(path)
    else:
        os.mkdir(path)

In [ ]:
datasets_base_path = "..\\..\\Dataset\\Clusters_CSV\\original\\"
dictionary_path = ".\\dictionary\\"
synonyms_path = ".\\synonyms\\"

In [ ]:
remove_and_make_dir(synonyms_path)

In [ ]:
cluster_dict = {}
for filename in os.listdir(dictionary_path):
    file_path = os.path.join(dictionary_path, filename)
    cluster = os.path.splitext(filename)[0]
    file = open(file_path)
    cluster_dict[cluster] = json.load(file)
    file.close()

In [ ]:
threshold = 0.0
inverted_index_cluster = {}
for cluster, inverted_index in cluster_dict.items():
    inverted_index_cluster[cluster] = {}
    for main_token, tokens in inverted_index.items():
        inverted_index_cluster[cluster][main_token] = [k for k,v in tokens.items() if v > threshold]

In [ ]:
cluster_synonyms = {}
for cluster, inverted_index in inverted_index_cluster.items():
    tmp = []
    for main_token, tokens in inverted_index.items():
        synonyms = set()
        synonyms.add(main_token)
        synonyms.update(tokens)
        for token in tokens:
            synonyms.update(inverted_index[token])
        tmp.append(synonyms)
    cluster_synonyms[cluster] = []
    for item in tmp:
        if item not in cluster_synonyms[cluster]:
            cluster_synonyms[cluster].append(item)
    cluster_synonyms[cluster] = [list(item) for item in cluster_synonyms[cluster]]

In [ ]:
# save synonyms
for cluster, synonyms in cluster_synonyms.items():
    json_obj = json.dumps(synonyms, indent=4) 
    f = open(synonyms_path+cluster+".txt","w")
    f.write(json_obj)
    f.close()    

In [ ]:
# read synonyms
cluster_synonyms = {}
for filename in os.listdir(synonyms_path):
    file_path = os.path.join(synonyms_path, filename)
    cluster = os.path.splitext(filename)[0]
    file = open(file_path)
    cluster_synonyms[cluster] = json.load(file)
    file.close()